In [ ]:
from pose import KeypointDetector
from classifier import PoseClassifier

detector = KeypointDetector()
classifier = PoseClassifier()

TEST_IMAGES_DIR = '../assets/images'

In [ ]:
import numpy as np
import os
from PIL import Image

image_path = os.path.join(TEST_IMAGES_DIR, 'take01.jpeg')

image = np.array(Image.open(image_path), dtype=np.uint8)

detection = detector.detect_keypoints(image)
print(f'Número de keypoints detectados: {len(detection)}')

prediction = classifier.predict([detection])

print(prediction)

img_with_keypoints = detector.draw_keypoints()

detector.display_image(img_with_keypoints)



In [ ]:
import cv2

cap = cv2.VideoCapture('../assets/videos/video.mp4')
frame_count = 0
detections = []
frame_interval = 1

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    if frame_count % frame_interval == 0:
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image = np.array(image, dtype=np.uint8)

        detection = detector.detect_keypoints(image)
        image = detector.draw_keypoints()
        detections.append(detection)

        if len(detections) >= frame_interval:
            batch_predictions = classifier.predict(detections)
            
            cv2.imshow('video', image)
            
            print("predictions:", batch_predictions)

            detections = []  

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    frame_count += 1

cap.release()
cv2.destroyAllWindows()

In [ ]:
import time
from models.vitpose.easy_ViTPose.vit_utils.inference import VideoReader
import tqdm
import cv2
from pose import KeypointDetector

wait = 0
total_frames = 1

input_path = '../assets/videos/video.mp4'

reader = VideoReader(input_path)
cap = cv2.VideoCapture(input_path) 
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
cap.release()
wait = 15

model = KeypointDetector(is_sequential=True)

print(f'>>> Running inference on {input_path}')
fps = []
tot_time = 0.
for (ith, img) in tqdm.tqdm(enumerate(reader), total=total_frames):
    frame_keypoints = model.detect_keypoints(img)
    image = model.draw_keypoints()
    model.display_image(image, wait=False)